# **Machine Learning**: Text-Klassifikation

Das Beispiel basiert auf einem [offenen Datensat](http://qwone.com/~jason/20Newsgroups/) von Newsgroup-Nachtrichten und orientiert sich an [diesem offiziellen Tutorial](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html) von scikit-learn zur Textanalyse.

Wir nutzen Dokumente von mehreren Newsgroups und trainieren damit einen Classifier, der dann eine Zudordnung von neuen Texten auf eine dieser Gruppen durchführen kann. Sprich die Newsgroups stellen die Klassen/Tags dar, mit denen wir neue Texte klassifizieren. Wie nutzen eine einfachen Bag-of-Word-Ansatz in dem wir (normalisierte) Häufigkeit von Wörtern als Features nutzen.

In diesem Fall liegen die Daten noch nicht als Teil von scikit-learn vor, es wird aber eine Funktion angeboten, mit die Daten bezogen werden können.

In [318]:
import numpy as np

from sklearn.datasets import fetch_20newsgroups
from pprint import pprint

## **20newsgroups** dataset

Wir legen vier Newsgroups, die wir nutzen wollen, fest.

In [287]:
selected_categories = ["sci.crypt", 
                       "sci.electronics", 
                       "sci.med", 
                       "sci.space"]

Wir beziehen die Trainingset- und Testsets-Dokumente.

In [288]:
newsgroup_posts_train = fetch_20newsgroups(
    data_home="newsgroup_data",
    subset='train',
    categories=selected_categories,
    shuffle=True,
    random_state=1)

newsgroup_posts_test = fetch_20newsgroups(
    data_home="newsgroup_data",
    subset='test',
    categories=selected_categories,
    shuffle=True,
    random_state=1)

Die Objekte, die wir erhalten, sind `scikit-learn-Bunches` …

In [289]:
type(newsgroup_posts_train)

sklearn.utils._bunch.Bunch

… und haben die üblichen Attribute von Bunches.

In [290]:
dir(newsgroup_posts_train)

['DESCR', 'data', 'filenames', 'target', 'target_names']

In [291]:
newsgroup_posts_test.target_names

['sci.crypt', 'sci.electronics', 'sci.med', 'sci.space']

U.a. exitiert die übliche Beschreibung des Datensets im Attribute DESCR, die wir uns ansehen können.

In [293]:
print(newsgroup_posts_train.DESCR)

.. _20newsgroups_dataset:

The 20 newsgroups text dataset
------------------------------

The 20 newsgroups dataset comprises around 18000 newsgroups posts on
20 topics split in two subsets: one for training (or development)
and the other one for testing (or for performance evaluation). The split
between the train and test set is based upon a messages posted before
and after a specific date.

This module contains two loaders. The first one,
:func:`sklearn.datasets.fetch_20newsgroups`,
returns a list of the raw texts that can be fed to text feature
extractors such as :class:`~sklearn.feature_extraction.text.CountVectorizer`
with custom parameters so as to extract feature vectors.
The second one, :func:`sklearn.datasets.fetch_20newsgroups_vectorized`,
returns ready-to-use features, i.e., it is not necessary to use a feature
extractor.

**Data Set Characteristics:**

=================   ==========
Classes                     20
Samples total            18846
Dimensionality               1

Das Attribut `data` enthält in diesem Fall keine Matrix, sondern Newsgroup-Message-Texte. Ein Beispiel schauen wir uns an:

In [294]:
print(newsgroup_posts_train.data[6])

From: pmetzger@snark.shearson.com (Perry E. Metzger)
Subject: Do we need the clipper for cheap security?
Organization: Partnership for an America Free Drug
Lines: 53

amanda@intercon.com (Amanda Walker) writes:
>> The answer seems obvious to me, they wouldn't.  There is other hardware 
>> out there not compromised.  DES as an example (triple DES as a better 
>> one.) 
>
>So, where can I buy a DES-encrypted cellular phone?  How much does it cost?
>Personally, Cylink stuff is out of my budget for personal use :)...

If the Clipper chip can do cheap crypto for the masses, obviously one
could do the same thing WITHOUT building in back doors.

Indeed, even without special engineering, you can construct a good
system right now. A standard codec chip, a chip to do vocoding, a DES
chip, a V32bis integrated modem module, and a small processor to do
glue work, are all you need to have a secure phone. You can dump one
or more of the above if you have a fast processor. With integration,
you could 

Die Targets sind die Newsgroup-Namen. Diese Klassen sind wie üblich für `scikit-learn` als Zahlen kodiert, die wir mittels `target_names` auflösen können.

In [295]:
print(newsgroup_posts_train.target_names)

['sci.crypt', 'sci.electronics', 'sci.med', 'sci.space']


In [296]:
newsgroup_posts_test.target[1050:1300]

array([3, 2, 0, 3, 3, 1, 0, 3, 2, 3, 0, 2, 2, 1, 1, 0, 1, 3, 1, 2, 0, 3,
       0, 2, 0, 2, 3, 2, 3, 2, 1, 1, 1, 2, 0, 0, 3, 1, 2, 2, 2, 0, 3, 0,
       0, 1, 0, 3, 1, 1, 2, 1, 3, 3, 1, 3, 1, 3, 1, 3, 2, 3, 3, 3, 2, 1,
       3, 0, 0, 0, 2, 2, 0, 3, 0, 1, 0, 2, 0, 2, 1, 0, 1, 2, 1, 0, 0, 0,
       3, 3, 0, 3, 1, 0, 3, 1, 1, 3, 1, 2, 0, 3, 0, 0, 0, 0, 0, 0, 1, 3,
       2, 0, 0, 2, 0, 2, 1, 1, 1, 2, 2, 0, 3, 3, 3, 1, 3, 3, 2, 1, 0, 3,
       2, 0, 1, 0, 0, 2, 0, 2, 0, 1, 2, 3, 3, 1, 2, 3, 2, 3, 3, 2, 1, 0,
       3, 1, 2, 1, 2, 2, 3, 0, 3, 0, 1, 0, 2, 0, 2, 0, 1, 3, 1, 0, 0, 3,
       3, 1, 3, 1, 2, 3, 0, 3, 1, 0, 1, 3, 2, 0, 2, 3, 3, 3, 3, 1, 0, 3,
       2, 2, 3, 3, 3, 0, 3, 2, 2, 1, 1, 0, 2, 0, 1, 0, 0, 2, 1, 0, 3, 0,
       1, 1, 3, 2, 2, 2, 2, 3, 2, 3, 3, 1, 0, 1, 0, 0, 3, 3, 0, 1, 1, 3,
       0, 2, 2, 0, 0, 0, 0, 1])

Für unsere Beispiel-Message:

In [297]:
newsgroup_posts_train.target_names[newsgroup_posts_train.target[6]]

'sci.crypt'

## **Feature Extraction**

Um die Wörter zu zählen, aber auch um Stopwörte zu entfernen und zu Tokenisieren nutzen wir ein Objekt der [CountVectorizer-Klasse](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) bzw. dessen `fit`-Methode

### **Tokenization**

`.fit()` method:

1. **Tokenization**: It breaks down the text into individual words or tokens based on a predefined separator (like spaces, punctuation, etc.). This process converts the raw text data into tokens or terms which are easier for the model to understand.

2. **Vocabulary Creation**: It scans through all the documents in newsgroup_posts_train.data to build a vocabulary of all unique tokens (words) present across the dataset. Each unique token is assigned a specific integer ID. This vocabulary is crucial for transforming the text documents into numerical data that machine learning algorithms can work with.

3. **Learn the Vocabulary**: The .fit() method only learns the vocabulary of the documents, meaning it identifies all unique words and their frequencies but does not transform the text data into numerical format. The actual conversion of text documents into a numerical matrix happens when you apply the .transform() method on the dataset.

4. **No Return Value**: The .fit() method updates the CountVectorizer instance (count_vect in your case) with the vocabulary. It doesn't return anything; instead, it prepares the CountVectorizer for the next step, which is usually transforming the documents using the .transform() method or directly converting to a document-term matrix with the .fit_transform() method.

In [298]:
from sklearn.feature_extraction.text import CountVectorizer

In [299]:
# Create instance of CountVectorizer()
count_vect = CountVectorizer()

# Fit a vocabulary with words (+ occurences) for all documents in newgroup_posts_train.data
count_vect.fit(newsgroup_posts_train.data)

CountVectorizer()

Über alle Dokumente bekommen wir die folgende Zusammenstellung der Wörter und ihre Indices (positionen im Array):

In [300]:
len(count_vect.get_feature_names_out())

38683

Wir können uns ein paar Beispiele ansehen …

In [301]:
count_vect.get_feature_names_out()[10000:10050]

array(['cellar', 'cellphone', 'cells', 'cellsat', 'cellular', 'cellulars',
       'celluloid', 'celp', 'celsius', 'cement', 'cen', 'censoring',
       'censorship', 'censure', 'census', 'cent', 'centaur', 'centauri',
       'centaurs', 'centennial', 'center', 'centered', 'centerline',
       'centerpiece', 'centers', 'centigrade', 'centimeter',
       'centimeters', 'central', 'centralia', 'centralised', 'centralism',
       'centralization', 'centralize', 'centralized', 'centrally',
       'centre', 'centres', 'centrifuge', 'centronic', 'cents', 'centure',
       'centuries', 'century', 'ceo', 'cepek', 'cephalopods', 'cept',
       'ceramic', 'cereal'], dtype=object)

… oder sogar das counting-Dictionary mit den Wörtern und ihre Vorkommen-Anzahl betrachten (Achtung: groß!).

In [302]:
len(count_vect.get_feature_names_out())

38683

In [303]:
count_vect.vocabulary_

{'from': 16874,
 'myers': 24949,
 'cs': 12139,
 'scarolina': 31323,
 'edu': 14486,
 'daniel': 12461,
 'subject': 33688,
 're': 29468,
 'is': 20559,
 'msg': 24737,
 'sensitivity': 31723,
 'superstition': 33952,
 'organization': 26440,
 'usc': 36540,
 'department': 12983,
 'of': 26126,
 'computer': 11168,
 'science': 31420,
 'lines': 22467,
 '39': 3170,
 'frequently': 16834,
 'late': 21996,
 'have': 18389,
 'been': 8093,
 'reacting': 29479,
 'to': 35157,
 'something': 32692,
 'added': 5849,
 'restaurant': 30292,
 'foods': 16578,
 'what': 37759,
 'happens': 18290,
 'that': 34796,
 'the': 34802,
 'inside': 20111,
 'my': 24940,
 'throat': 34979,
 'starts': 33253,
 'feel': 16076,
 'puffy': 28922,
 'like': 22414,
 'cold': 10827,
 'and': 6641,
 'also': 6422,
 'at': 7375,
 'times': 35085,
 'mouth': 24652,
 'especially': 15285,
 'tongue': 35230,
 'lips': 22499,
 'situations': 32294,
 'around': 7133,
 'these': 34874,
 'symptoms': 34218,
 'almost': 6396,
 'always': 6466,
 'involve': 20469,
 'resta

### `tf` / `tfidf` **Frequencies**

Diese Countings müssen wir für den Klassifikator in eine Matrix transformieren:

In [304]:
# Transfrom vocab into sparse matrix
X_train_counts = count_vect.transform(newsgroup_posts_train.data)

Die Matrix, die wir erhalten, hat folgende Maße:

In [305]:
X_train_counts.shape

(2373, 38683)

Wir normalisieren die Wörtercoutings auf die Anzahl an Wörter im Text (Term Frequency - TF). Dazu nutzen wir eine Objekt der Klasse [TfidfTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html) (schalten die idf-Normalisierung (Inverse Document Frequency) dabei ab.)

In [306]:
from sklearn.feature_extraction.text import TfidfTransformer

In [307]:
tf_transformer = TfidfTransformer(use_idf=False)

Die Normalisierung erfolgt mit den Methoden `fit` und `transform`.

In [308]:
tf_transformer.fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)

Die Matrix, die wir erhalten, hat folgende Maße:

In [309]:
X_train_tf.shape

(2373, 38683)

### **Classifier Training**

Jetzt können wir einen Klassifikator erstellen. Wir Nutzen hier eine Random-Forest-Klassifikator, könnten aber auch eine andere Methode wählen.

#### **Random Forest** Classifier

In [234]:
from sklearn.ensemble import RandomForestClassifier

In [235]:
tf_random_forest_classifier = RandomForestClassifier()

Wie bei allen Supervised-Learning-Verfahren trainieren wir den Klassikator mit der Trainingsmatrix.

In [236]:
tf_random_forest_classifier.fit(X_train_tf,
                                newsgroup_posts_train.target)

RandomForestClassifier()

Um zu testen wie gut der Klassifikator funktioniert, prozessieren wir das Test-Set mit dem CountVectorizer-Objekt und führen die gleiche TF-Transformation durch.

In [237]:
X_test_counts = count_vect.transform(newsgroup_posts_test.data)
X_test_tf = tf_transformer.transform(X_test_counts)

Ein kurze Blick auf die Maße der Matrix, zeigt uns, dass die Anzahl an Spalten (Features) gleich ist wie bei der Trainingsmatrix.

In [239]:
X_test_counts.shape

(1579, 38683)

Jetzt können wir mit der score-Methods die Güte des Klassikators auf dem Test-Set prüfen.

In [240]:
tf_random_forest_classifier.score(X_test_tf, 
                                  newsgroup_posts_test.target)

0.8556048131728943

Der Klassifikator scheint gut genug zu funktionieren. Wir können jetzt Listen von Dokumenten klassifizieren. Wir nehmen zwei Dokumete aus unserem Test-Set und erstellen zusätzlich ein sehr kleines eigene Dokument, das nur aus einem Satz bestehent.

In [241]:
docs_to_classify = [
    newsgroup_posts_test.data[1],
    newsgroup_posts_test.data[7],
    "The sun send a lot of radiation to the planets including earth"]

Werfen wir einen kurzen Blick auf die zwei Dokumente aus dem Testset.

In [244]:
for d in docs_to_classify:
    print(d[:100])

From: dmuntz@quip.eecs.umich.edu (Dan Muntz)
Subject: Re: new encryption
Organization: University of
From: jcarey@news.weeg.uiowa.edu (John Carey)
Subject: med school
Organization: University of Iowa, 
The sun send a lot of radiation to the planets including earth


Auch diese neu zu klassifizierenden Dokumente müssen wir wie die Traininsdokumente in Matrizen transformieren:

In [245]:
X_to_classify_counts = count_vect.transform(docs_to_classify)
X_to_classify_tfidf = tf_transformer.transform(X_to_classify_counts)

Jetzt können wir mit dieser Matrix die Klassifikation durchführen …

In [246]:
predicted_classes = tf_random_forest_classifier.predict(X_to_classify_tfidf)

… und uns die Klassen anschauen, mit denen die Dokumente versehen wurden.

In [247]:
for pclass in predicted_classes:
    print(newsgroup_posts_train.target_names[pclass])

sci.crypt
sci.med
sci.electronics


## **Aufgabe**: TFIDF-Transformer + weitere Modelle

In [34]:
import numpy as np

# Data set
from sklearn.datasets import fetch_20newsgroups

# Tokenizer / Vectorizer / Transformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# ML models
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

# Helper functions
from sklearn.pipeline import Pipeline

### **Load data**
Select training categories and load 20newsgroups data set

In [53]:
categories = ["sci.crypt", "sci.electronics", "sci.med", "sci.space"]

news20_train = fetch_20newsgroups(data_home='newsgroup_data',
                                  subset='train',
                                  categories=categories,
                                  shuffle=True,
                                  random_state=42)

news20_test = fetch_20newsgroups(data_home='newsgroup_data',
                                 subset='test',
                                 categories=categories,
                                 shuffle=True,
                                 random_state=42)

docs_for_testing = ['CRISPR gene editing represents a groundbreaking advancement in genetic engineering, offering unprecedented capabilities for targeted genome modifications with far-reaching implications across various fields of science and medicine.',
                    'The moon has a mean radius of 1,079.6 miles (1,737.5 kilometers) and a mean diameter of 2,159.2 miles (3,475 km). It is less than a third the width of Earth, making it about one-quarter (27 percent) the size of our planet.',
                    'A secure password is one that is sufficiently long, complex, and unique to prevent unauthorized access to accounts. It should be at least 12 characters long, but longer is better, and include a mix of uppercase letters, lowercase letters, numbers, and symbols. Avoid using common words, personal information, or easily guessable sequences. Strong passwords can be in the form of passphrases that are easier to remember but difficult to crack.',
                    'Der Schutz vertraulicher Daten und der persönlichen Identität spielt im Zeitalter der Vernetzung und des E-Commerce eine zentrale Rolle sowohl für Einzelpersonen als auch für Unternehmen in allen Größen. Die angewandte Kryptographie spielt dabei eine zentrale Rolle. Sie umfasst die Themen Verschlüsselung, Public-Key-Kryptographie, Authentifikation, digitale Signatur, elektronisches Bargeld, Blockchain-Technologie und sichere Netze.',
                    'In seinem letzten Buch gibt Stephen Hawking Antworten auf die drängendsten Fragen unserer Zeit und nimmt uns mit auf eine persönliche Reise durch das Universum seiner Weltanschauung. Seine Gedanken zu Ursprung und Zukunft der Menschheit sind zugleich eine Mahnung, unseren Heimatplaneten besser vor den Gefahren unserer Gegenwart zu schützen.']

### **TFIDF-Transformer + Random Forest**

#### Step 1: **Feature Extraction**

In [54]:
# Tokenize raw documents and learn token counts via CountVectorizer().fit
cvec = CountVectorizer().fit(news20_train.data)

In [55]:
len(cvec.vocabulary_)

38683

#### Step 2: Transform token counts into `tf` / `tfidf` **Frequencies**

In [56]:
# Create a tfidf transformer
tfidf_converter = TfidfTransformer(use_idf=True)

# Transform vocabs into sparse matrix for both subsets
X_train_m = cvec.transform(news20_train.data)
X_test_m = cvec.transform(news20_test.data)

# Learn tfidf vectors for training subset
tfidf_converter.fit(X_train_m)

# Transform count matrix to tf / tfidf frequency matrix
X_train_tfidf = tfidf_converter.transform(X_train_m)
X_test_tfidf = tfidf_converter.transform(X_test_m)

#### Step 3: Train the **Classifier**

In [57]:
# Create a classifier object
tfidf_randomforest_clf = RandomForestClassifier()

# Train the classifier with tfidf data and target categories
tfidf_randomforest_clf.fit(X_train_tfidf, news20_train.target)

RandomForestClassifier()

#### Step 4: Evaluate the **Classifier**

In [58]:
# Evaluation on test subset
tfidf_randomforest_clf.score(X_test_tfidf, news20_test.target)

0.8606713109563014

In [64]:
# Evaluate on docs_for_testing data
docs_for_testing_c = cvec.transform(docs_for_testing)
docs_for_testing_tfidf = tfidf_converter.transform(docs_for_testing_c)

predicted = tfidf_randomforest_clf.predict(docs_for_testing_tfidf)

for doc, cat in zip(docs_for_testing, predicted):
    print(f'{doc[:40]} --> {news20_train.target_names[cat]}') 

CRISPR gene editing represents a groundb --> sci.electronics
The moon has a mean radius of 1,079.6 mi --> sci.space
A secure password is one that is suffici --> sci.crypt
Der Schutz vertraulicher Daten und der p --> sci.electronics
In seinem letzten Buch gibt Stephen Hawk --> sci.electronics


### **TFIDF-Transformer + Naive Bayes** Classifier

#### Train the **Classifier**

In [68]:
# Create a classifier object
tfidf_nbayes_clf = MultinomialNB()

# Train the classifier with tfidf data and target categories
tfidf_nbayes_clf.fit(X_train_tfidf, news20_train.target)

MultinomialNB()

#### Evaluate the **Classifier**

In [69]:
# Evaluation on test subset
tfidf_nbayes_clf.score(X_test_tfidf, news20_test.target)

0.8853704876504117

In [70]:
# Evaluate on docs_for_testing data
predicted = tfidf_nbayes_clf.predict(docs_for_testing_tfidf)

for doc, cat in zip(docs_for_testing, predicted):
    print(f'{doc[:40]} --> {news20_train.target_names[cat]}') 

CRISPR gene editing represents a groundb --> sci.med
The moon has a mean radius of 1,079.6 mi --> sci.space
A secure password is one that is suffici --> sci.crypt
Der Schutz vertraulicher Daten und der p --> sci.crypt
In seinem letzten Buch gibt Stephen Hawk --> sci.crypt


### **TFIDF-Transformer + SVMachines** Classifier

#### Train the **Classifier**

In [71]:
# Create a classifier object
tfidf_svmachine_clf = SGDClassifier(loss='hinge', penalty='l2',
                                     alpha=1e-3, random_state=42,
                                     max_iter=5, tol=None)

# Train the classifier with tfidf data and target categories
tfidf_svmachine_clf.fit(X_train_tfidf, news20_train.target)

SGDClassifier(alpha=0.001, max_iter=5, random_state=42, tol=None)

#### Evaluate the **Classifier**

In [72]:
# Evaluation on test subset
tfidf_svmachine_clf.score(X_test_tfidf, news20_test.target)

0.9379354021532615

In [73]:
# Evaluate on docs_for_testing data
predicted = tfidf_svmachine_clf.predict(docs_for_testing_tfidf)

for doc, cat in zip(docs_for_testing, predicted):
    print(f'{doc[:40]} --> {news20_train.target_names[cat]}') 

CRISPR gene editing represents a groundb --> sci.med
The moon has a mean radius of 1,079.6 mi --> sci.space
A secure password is one that is suffici --> sci.crypt
Der Schutz vertraulicher Daten und der p --> sci.crypt
In seinem letzten Buch gibt Stephen Hawk --> sci.electronics


### **TFIDF-Transformer + Neural Network** Classifier

#### Train the **Classifier**

In [74]:
# Create a classifier object
tfidf_nn_clf = MLPClassifier()

# Train the classifier with tfidf data and target categories
tfidf_nn_clf.fit(X_train_tfidf, news20_train.target)

MLPClassifier()

#### Evaluate the **Classifier**

In [75]:
# Evaluation on test subset
tfidf_nn_clf.score(X_test_tfidf, news20_test.target)

0.9303356554781508

In [76]:
# Evaluate on docs_for_testing data
predicted = tfidf_nn_clf.predict(docs_for_testing_tfidf)

for doc, cat in zip(docs_for_testing, predicted):
    print(f'{doc[:40]} --> {news20_train.target_names[cat]}') 

CRISPR gene editing represents a groundb --> sci.med
The moon has a mean radius of 1,079.6 mi --> sci.space
A secure password is one that is suffici --> sci.crypt
Der Schutz vertraulicher Daten und der p --> sci.crypt
In seinem letzten Buch gibt Stephen Hawk --> sci.electronics
